# **Projet P6 - révisions et pratique**

Vous allez travailler sur des données extraites de IMDB. Cela vous permettra si vous les souhaitez (plus tard!) d'inclure ce travail à votre projet TheMoviePredictor que vous faites avec Arnaud dans lequel vous récupérez justement ces données et construisez votre base. La variable d'intérêt sera la notation IMDB des films pour pouvoir déterminer. En effet le succès commercial d'un film n'implique pas nécessairement sa qualité et il convient donc d'aller chercher plus loin que le simple profit dégagé d'une production cinématographique...
À vous !!

Les **objectifs** de ce projet sont multiples :
1. Réviser
2. Pratiquer
3. Vous auto-évaluer et vous évaluer (pour nous)
4. Vous rassurer et vous permettre de réaliser ce que vous savez faire pour pouvoir en parler

1. [Import des librairies](#import_lib)<br>
2. [Import des données](#import_data)<br>
3. [Nettoyage des donnéees](#data_cleaning)<br>
4. [Analyse exploratoire](#exploration)<br>
5. [Pré-traitement](#preprocess)<br>
6. [Une régression linéaire](#reglin)<br>
7. [D'autres modèles de régression](#autres_reg)<br>
8. [De la régression à la classification](#reg_to_class)<br>
9. [Une régression logistique](#reglog)<br>
10. [D'autres modèles de classification](#autre_class)<br>
11. [En option](#option)<br>
    11.1 [Un outil de recommandation](#reco)<br>
    11.2 [Sauvegarder un modèle](#save)<br>
    11.3 [Analyse en composantes principales](#acp)<br>

<a id='import_lib'></a>

## **1. Import des libraries**

**À FAIRE**

> Importer dans la cellule l'ensemble des librairies nécessaires à votre travail. L'idée n'est pas de savoir immédiatement tout ce dont vous aurez besoin mais de faire des aller-retours pour y ajouter vos librairies petit à petit. L'intérêt est une meilleure lisibilité pour un lecteur extérieur qui, en quelques lignes d'import, pourra déjà avoir une idée de ce qui a été fait.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from statsmodels.api import OLS, add_constant

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE, RFECV
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.svm import SVC, SVR
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, cross_validate
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler

from xgboost import XGBRegressor, XGBClassifier

<a id='import_data'></a>

## **2. Import des données**

**À FAIRE**

> Importer les données `5000_movies_bis.csv` disponible à la racine de ce document.  
> Afficher les 7 premières lignes et **toutes** les colonnes.    
> Répondre aux questions suivantes (répondez à toutes les questions dans une seule cellule Markdown mais évidemment le code vous ayant permis d'extraire ces informations doit être présent):
>- combien y a-t-il d'observations/de variables ?
>- sur combien d'années se répartissent les données ?
>- combien de pays sont représentés ?
>- combien de réalisateurs différents dans la base ?
>- combien d'acteurs et d'actrices différentes ?

In [ ]:
data = pd.read_csv('5000_movies_bis.csv')
data.head(7)

In [ ]:
# pour afficher toutes les colonnes on peut gérer les options d'affichage de pandas
pd.set_option('display.max_columns', 100)
data.head(7)

In [ ]:
data.shape

In [ ]:
data.director_name.unique().shape

In [ ]:
max(data.title_year)-min(data.title_year)

In [ ]:
data.country.nunique(dropna=True)

In [ ]:
actors = pd.Series(data[['actor_1_name','actor_2_name','actor_3_name']].values.flatten())
actors.nunique(dropna=True)

<a id='data_cleaning'></a>

## **3. Nettoyage des données**

**À FAIRE**

> Vous allez dans cette partie vous occuper de faire les opérations de nettoyage sur les données. Cela implique donc de regarder en détail :
>- les doublons
>- les variables (à supprimer, à modifier etc...)
>- les valeurs manquantes
>- les zéros
> 
>Ajouter une courte explication des décisions que vous prendrez (gestion des valeurs manquantes, suppression ou modification de certaines variables, etc...).

#### **Les doublons**

In [ ]:
sum(data.duplicated())

In [ ]:
data.drop_duplicates(inplace=True)
data.shape

In [ ]:
# Visualisation des doublons
data = data.sort_values(by=['movie_title','num_voted_users'], ascending=[True,False])

data[data.groupby('movie_title')['movie_title'].transform('size') > 1]#.head()

In [ ]:
# Suppression des doublons
data.drop_duplicates(subset=['director_name', 'movie_title', 'title_year'], keep="first", inplace=True)
len(data)

#### **Suppression de variables**

In [ ]:
data.columns

La variable `plot_keywords` pourrait être utile mais sera compliquée à gérer.  
La variable `movie_imdb_link` n'a aucun intérêt pour nous ici. Enfin elle a quand même servi pour aller scraper quelques infos supplémentaires sur IMDB mais maintenant que c'est fait, plus besoin.  
On peut donc d'ores et déjà se séparer de ces 2 variables.

In [ ]:
data.drop(['plot_keywords', 'movie_imdb_link'], axis=1, inplace=True)
data.shape

#### **Les valeurs manquantes et les zéros**

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(18,8))
sns.heatmap(data.isnull(), yticklabels=False, cbar=False, cmap='inferno', ax=ax[0])
ax[0].set_title('Les NaNs')
sns.heatmap(data==0, yticklabels=False, cbar=False, cmap='inferno', ax=ax[1])
ax[1].set_title('Les zéros');

**Gestion des zéros**

La plupart des zéros sont des valeurs manquantes vraisemblablement, à l'exception de ceux de la variable `facenumber_in_poster`. On les remplace par des NaN.

In [ ]:
#sélection et affichage des variables contenant des 0 sauf facenumber_in_poster
cols = data.columns[(data==0).any()].drop('facenumber_in_poster')

#remplacement des 0 par des nan
data[cols] = data[cols].replace(0, np.nan)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(18,8))
sns.heatmap(data.isnull(), yticklabels=False, cbar=False, cmap='inferno', ax=ax[0])
ax[0].set_title('Les NaNs')
sns.heatmap(data==0, yticklabels=False, cbar=False, cmap='inferno', ax=ax[1])
ax[1].set_title('Les zéros');

**Gestion des NaN**

In [ ]:
data.isnull().sum().sort_values(ascending=False)/len(data)

`movie_fb_likes` et `director_fb_likes` ont trop de valeurs manquantes et supprimer les lignes diminuerait de manière trop importante la taille de notre dataset. On ne conserve donc pas ces variables.

In [ ]:
# suppression 
data.drop(['movie_fb_likes', 'director_fb_likes'], axis=1, inplace=True)
data.shape

**Dans le dataset initial (5000_movies.csv)**, il y a beaucoup de valeurs manquantes dans `gross` et dans `budget` donc imputation délicate mais comme on veut garder ces variables, on va alors supprimer les lignes. Avant cette suppression on peut essayer de scraper IMDB pour obtenir des informations supplémentaires. Pour cela, un petit notebook séparé dans le même dossier permet de récupérer les données supplémentaires éventuelles, de les ajouter dans les colonnes `gross` et `budget` du dataframe `data`. Comme l'éxecution est un peu longue, on va sauvegarder ce dataframe dans un nouveau csv pour ne pas avoir à le refaire à chaque fois.

In [ ]:
data.dropna(subset=['gross', 'budget'], inplace=True)
data.shape

In [ ]:
# Pourcentage d'observations écartées :
round((4919-4060)/4919,2)

In [ ]:
# nombre d'observations encore incomplètes: 
4213-data.dropna().shape[0]

`aspect_ratio` est la 3ème variable avec le plus de NaN. Une rapide recherche et on trouve que c'est le rapport de la largeur sur la hauteur de l'image. Probablement un intérêt limité dans notre cas mais on vérifie avant de décider si on la conserve.

In [ ]:
data.aspect_ratio.value_counts()

In [ ]:
# Principalement 2.35 et 1.85, on regroupe les autres valeurs sous une seule modalité et on va calculer la moyenne et la variance du score de chaque sous-groupe
print(
    'Les moyennes des 3 groupes sont :',
    data.imdb_score[data.aspect_ratio==2.35].mean(),
    data.imdb_score[data.aspect_ratio==1.85].mean(),
    data.imdb_score[(data.aspect_ratio!=2.35)&(data.aspect_ratio!=1.85)].mean()
)

print(
    'Les variances des 3 groupes sont :',
    data.imdb_score[data.aspect_ratio==2.35].std()**2,
    data.imdb_score[data.aspect_ratio==1.85].std()**2,
    data.imdb_score[(data.aspect_ratio!=2.35)&(data.aspect_ratio!=1.85)].std()**2
)

In [ ]:
# Étant données les moyennes et variances très proches des 3 groupes, on peut supprimer cette variable de notre analyse sans que cela affecte les résultats.
data.drop('aspect_ratio', axis=1, inplace=True)
data.shape

`content_rating` est la 4ème variable avec le plus de NaN. On souhaite conserver cette variable en revanche on pourra difficilement faire de l'imputation de valeurs qui a du sens donc on choisit de supprimer les lignes.

In [ ]:
data.dropna(subset=['content_rating'], inplace=True)
data.shape

In [ ]:
data.isnull().sum().sort_values(ascending=False)

On peut regarder certaines variables "à la main" car on pourra trouver les informations facilement.

In [ ]:
# facenumber_in_poster
data[data.facenumber_in_poster.isna()]

In [ ]:
# il y en a que 9, exceptionnellement on peut aller voir les affiches des films ça va aller vite
data.loc[99,'facenumber_in_poster'] = 1 #hobbit
data.loc[248,'facenumber_in_poster'] = 8 #tortues ninja
data.loc[1948,'facenumber_in_poster'] = 2 #dear john
data.loc[3016,'facenumber_in_poster'] = 2 #heaven
data.loc[3373,'facenumber_in_poster'] = 0 #kicks
data.loc[3797,'facenumber_in_poster'] = 0 #the visit
data.loc[3853,'facenumber_in_poster'] = 6 #mom's night out
data.loc[4444,'facenumber_in_poster'] = 4 #Growing Up Smith
data.loc[4692,'facenumber_in_poster'] = 4 #The Sisterhood of Night

In [ ]:
# color
data[data.color.isna()]

In [ ]:
# les 4 valeurs manquantes correspondent à des films récents et en couleur
data.loc[data.color.isna(), 'color'] = 'Color'

In [ ]:
# language
data[data.language.isna()]

In [ ]:
# les 3 valeurs manquantes correspondent à des films américains
data.loc[data.language.isna(), 'language'] = 'English'

In [ ]:
#title_year
data[data.title_year.isna()]

In [ ]:
# 1 valeur manquante pour le film Carlos qui date de 2010
data.loc[2466, 'title_year'] = 2010

On regarde un peu plus en détails les histoires de likes facebook. Notamment la correlation entre ces variables.

In [ ]:
data[['cast_total_fb_likes', 'actor_1_fb_likes', 'actor_2_fb_likes', 'actor_3_fb_likes', 'imdb_score']].corr()

`cast_total_fb_likes` et `actor_1_fb_likes` sont très corrélées. Pour les 2 autres variables `actor_2_fb_likes` et `actor_3_fb_likes` on peut choisir de les supprimer pour les remplacer par une information `other_actors_fb_likes` = `cast_total_fb_likes` - `actor_1_fb_likes`, cela permettra de diminuer le nombre de valeurs manquantes.

In [ ]:
# calcul de other_actors_fb_likes
data['other_actors_fb_likes'] = data.cast_total_fb_likes - data.actor_1_fb_likes

In [ ]:
# suppression variables redondantes
data.drop(['cast_total_fb_likes', 'actor_2_fb_likes', 'actor_3_fb_likes'], axis=1, inplace=True)
data.shape

In [ ]:
data.isnull().sum().sort_values(ascending=False)

Finalement, pour les 6 lignes ayant des valeurs manquantes restantes pour les likes facebook on peut imputer la moyenne.  
Idem pour num_critic_for_reviews, num_user_for_reviews et duration.  
Quant aux noms d'acteurs/réalisateurs, comme on ne les gardera pas pour la partie modélisation mais uniquement pour la visualisation (on reviendra dessus le moment venu), on peut les conserver tels quels pour le moment.

In [ ]:
#imputation de la moyenne pour quelques valeurs manquantes
data.loc[data.other_actors_fb_likes.isna(),'other_actors_fb_likes'] = data.other_actors_fb_likes.mean()
data.loc[data.actor_1_fb_likes.isna(),'actor_1_fb_likes'] = data.actor_1_fb_likes.mean()
data.loc[data.num_critic_for_reviews.isna(),'num_critic_for_reviews'] = data.num_critic_for_reviews.mean()
data.loc[data.num_user_for_reviews.isna(),'num_user_for_reviews'] = data.num_user_for_reviews.mean()
data.loc[data.duration.isna(),'duration'] = data.duration.mean()

In [ ]:
# on reset l'index après toutes les suppressions de lignes
data.reset_index(drop=True, inplace=True)

#### **Modification de variables**

In [ ]:
data.head()

In [ ]:
# affichage des valeurs pour voir s'il n'y a pas de caractères spéciaux qui se baladent.
data.loc[64].values

In [ ]:
# c'est pas évident à voir mais il y a bien un \xa0 louche après les titres des films
data.movie_title = data.movie_title.apply(lambda row : row.replace('\xa0',''))

On va regarder plus en détail la variable `content_rating`.

In [ ]:
data.content_rating.value_counts()

Une petite recherche et on trouve que historiquement, on a plus ou moins :
- Passed = Approved = M = GP = PG
- TV-14 = PG-13
- X = NC-17
- Not Rated = Unrated = NR

On veut donc remplacer :
- Passed, Approved, M et GP par **PG**
- TV-14 par **PG-13**
- X par **NC-17**
- Not Rated et Unrated par **UR** qui sont les notations utilisées aujourd'hui.

In [ ]:
def replace_rating(row):
    if row['content_rating'] in ['Passed', 'Approved', 'M', 'GP']:
        return 'PG'
    elif row['content_rating'] in ['Not Rated', 'Unrated']:
        return 'UR'
    elif row['content_rating'] == 'X':
        return 'NC-17'
    elif row['content_rating'] == 'TV-14':
        return 'PG-13'
    else:
        return row['content_rating']
    
data['content_rating'] = data.apply(replace_rating, axis=1)
data.content_rating.value_counts()

On a déjà géré presque tout, le dernier point en suspens est la variable `genre` qu'il faut spliter puis créer des OneHotEncoded variables.

In [ ]:
genre_dummies = data.genres.str.get_dummies('|')
data = pd.concat([data,genre_dummies], axis=1)
data.drop(['genres'], axis=1, inplace=True)
data

In [ ]:
data.language.value_counts()

In [ ]:
def replace_language(row):
    if row['language'] in ['French', 'Spanish', 'German', 'Italian', 'Portuguese', 'Norwegian', 'Dutch',
                           'Danish', 'Romanian', 'Bosnian', 'Czech', 'Hungarian', 'Swedish']:
        return 'European'
    elif row['language'] == 'English':
        return 'English'
    else:
        return 'Other languages'
    
data['language'] = data.apply(replace_language, axis=1)
data.language.value_counts()

In [ ]:
data.country.value_counts()

In [ ]:
def replace_country(row):
    if row['country'] in ['UK', 'France', 'Spain', 'Germany', 'West Germany', 'Italy', 'Portugal', 'Norway', 'Netherlands',
                        'Denmark', 'Ireland', 'Romania', 'Iceland', 'Czech', 'Hungary', 'Sweden', 'Belgium', 'Greece',
                        'Bulgaria', 'Switzerland', 'Poland', 'Finland']:
        return 'Europe'
    elif row['country'] in ['USA', 'Canada']:
        return 'North America'
    else:
        return 'Other countries'
    
data['country'] = data.apply(replace_country, axis=1)
data.country.value_counts()

## **4. Analyse exploratoire**

<a id='exploration'></a>

Dans cette partie, vous devez "explorer" vos données. Cette tâche, qui peut s'avérer très vaste, consiste à s'intéresser à l'information contenue dans nos données "au premier abord".

Sont donc attendus dans cette partie :
>- quelques statistiques descriptives
>- entre 6 et 10 visualisations (vous pouvez bien sûr en regrouper plusieurs sur une même figure)
>- et pour chaque résultat/graphique présenté, une explication succinte

In [ ]:
data.describe()

In [ ]:
#historique des sorties de films
data.hist('title_year', bins=45);

In [ ]:
#Les moyennes par genre
mean_genre = {}
for gen in data.columns[-21:]:
    mean_genre[gen] = data.loc[data[gen]==1,'imdb_score'].mean()
    
plt.bar(mean_genre.keys(), mean_genre.values())
plt.xticks(rotation=90);

On peut regarder un peu le profit généré par les films pour cela on crée la variable `profit` = `gross` - `budget`

In [ ]:
#Top 20 des films ayant généré le plus de profit
data['profit'] = data.gross - data.budget
top20profit = data[['movie_title', 'profit', 'budget']].sort_values('profit', ascending=False).iloc[:20].reset_index(drop=True)
top20profit

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(23,20))
top20profit.plot('budget', 'profit', kind='scatter', ax=ax, s=100, linewidth=0, c=range(20), colormap="plasma", colorbar=False);

for k in range(20):
    ax.annotate(top20profit.loc[k,'movie_title'], top20profit.loc[k,['budget','profit']],
                xytext=(10,-5), textcoords='offset points',
                family='sans-serif', fontsize=13, color='darkslategrey')

In [ ]:
#Retour sur investissement du top20 des profits
top20profit['roi'] = top20profit.profit/top20profit.budget*100
top20profit = top20profit.sort_values('roi', ascending=False).reset_index(drop=True)

fig, ax = plt.subplots(1,1,figsize=(15,12))
top20profit.plot('budget', 'roi', kind='scatter', ax=ax, s=100, linewidth=0, c=range(20), colormap="plasma", colorbar=False);
for k in range(20):
    ax.annotate(top20profit.loc[k,'movie_title'], top20profit.loc[k,['budget','roi']],
                xytext=(10,-5), textcoords='offset points',
                family='sans-serif', fontsize=13, color='darkslategrey')

In [ ]:
#Top 10 des films ayant les meilleurs retour sur investissement
data['roi'] = data.profit/data.budget*100
top10roi = data[['movie_title', 'roi', 'budget']].sort_values('roi', ascending=False).iloc[:10].reset_index(drop=True)

fig, ax = plt.subplots(1,1,figsize=(15,12))
top10roi.plot('budget', 'roi', kind='scatter', ax=ax, s=100, linewidth=0, c=range(10), colormap="plasma", colorbar=False);
for k in range(10):
    ax.annotate(top10roi.loc[k,'movie_title'], top10roi.loc[k,['budget','roi']],
                xytext=(10,-5), textcoords='offset points',
                family='sans-serif', fontsize=13, color='darkslategrey')

In [ ]:
#Top 20 des réalisateurs ayant les meilleurs notes IMDB et leur nombre de films dans la base
directors = data[['director_name', 'imdb_score', 'movie_title']].groupby('director_name').agg({'imdb_score':'mean', 'movie_title':'count'})
top20real = directors.sort_values('imdb_score', ascending=False).iloc[:20]
top20real

<a id='preprocess'></a>

## **5. Pré-traitement**

**À FAIRE**

>Maintenant que vous commencez à bien connaître votre base de données, on va la préparer pour la partie modélisation.
>
>Sont donc attendus dans cette partie :
>- restriction aux données utiles à la prédiction : potentiellement certaines variables conservées pour la visualisation sont à supprimer pour la modélisation
>- création des échantillons d'entraînement et de test
>- gestion des variables catégoriques d'un côté et numériques de l'autre
>
>La standardisation n'étant pas toujours nécessaire puisque ça dépend des modèles, vous pouvez choisir de la faire dès maintenant ou bien d'attendre de voir si vous en avez besoin...

**Suppression de variables**

Pour commencer on peut ensuite s'interroger sur l'intérêt de conserver les noms des acteurs/réalisateurs dans les variables car il y en a énormément (2399 réalisateurs et 6256 acteurs différents). Cette très forte variabilité limite l'impact de ces variables on va donc supprimer ces variables.

On peut aussi déjà virer la variable `profit` qu'on a créé pour la partie visualisation.

In [ ]:
data.drop(['director_name', 'actor_2_name', 'actor_1_name', 'movie_title', 'actor_3_name', 'profit', 'roi'], axis=1, inplace=True)
data.shape

**Variables catégoriques**

In [ ]:
data.select_dtypes(include="object")

In [ ]:
# On va maintenant créer les dummy variables
for varcat in data.select_dtypes(include="object").columns :
    dum = pd.get_dummies(data[varcat], drop_first=True)
    data = pd.concat([data,dum], axis=1)
    data.drop([varcat], axis=1, inplace=True)
data

**Variables numériques**

In [ ]:
data.describe()

Comme on va utiliser vraisemblablement différents modèles dont certains nécéssitent un *feature scaling*, on va le faire dès maintenant (enfin une fois qu'on aura découpé nos données en échantillons *train* et *test* puisque, pour rappel, les données *test* doivent être transformé de la même manière que les données *train*)

**Création des échantillons pour la modélisation**

In [ ]:
# matrice X et vecteur y
X = data.drop('imdb_score', axis=1)
y = data['imdb_score']

X.head()

<a id='reglin'></a>

## **6. Une régression linéaire**

**À FAIRE**

>Tout est dans le titre. Vous devez ici entraîner et tester une régression linéaire pour la prédiction de la note IMDB.  
>Par ailleurs, sont attendus ici :
>- un affichage et une interprétation des coefficients et de leur significativité
>- le choix d'une mesure d'évaluation du modèle et son interprétation
>- une validation croisée pour l'estimation de la qualité du modèle
>- *facultatif : l'ajout d'une régularisation Ridge ou Lasso pour déterminer si les résultats sont meilleurs*

In [ ]:
# échantillons train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
#feature scaling
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_train.columns, index = X_test.index)

In [ ]:
# pour pouvoir obtenir simplement les coefficients et les p-values, on passe par OLS de statsmodel
X_train_const = add_constant(X_train) # pour ajouter une constante à notre modèle sinon il n'y en a pas par défaut

# on fit la régression linéaire sur les données d'entrainement et on affiche direct le summary
OLS(y_train, X_train_const).fit().summary()

In [ ]:
# MSE
mean_squared_error(OLS(y_train,X_train_const).fit().predict(X_train_const),y_train)

In [ ]:
# MSE sur X_test
X_test_const = add_constant(X_test)
mean_squared_error(OLS(y_train,X_train_const).fit().predict(X_test_const),y_test)

In [ ]:
# On peut aussi utiliser RFE pour sélection les variables : disons qu'on veut en garder en 20
rfe = RFE(LinearRegression(), 20)
rfe = rfe.fit(X_train, y_train)

In [ ]:
col_rfe = X_train.columns[rfe.support_]
col_rfe

In [ ]:
#colonnes supprimées
X_train.columns[~rfe.support_]

In [ ]:
#On crée un jeu de données RFE avec uniquement les variables sélectionnées via RFE
X_train_rfe = X_train[col_rfe]
X_test_rfe = X_test[col_rfe]

In [ ]:
OLS(y_train, add_constant(X_train_rfe)).fit().summary()

In [ ]:
# MSE sur X_train_rfe
print('train', mean_squared_error(OLS(y_train,add_constant(X_train_rfe)).fit().predict(add_constant(X_train_rfe)),y_train))
print('test', mean_squared_error(OLS(y_train,add_constant(X_train_rfe)).fit().predict(add_constant(X_test_rfe)),y_test))

Comme on pouvait s'y attendre, la régression linéaire n'est vraiment pas terrible puisque le modèle n'est pas linéaire...on peut essayer une regression régularisée mais on est pas non plus en situation de sur-apprentissage donc l'intérêt est limité. Pour les avoir testées, les régressions Ridge et Lasso ne donnent pas non plus de bons résultats.

<a id='autres_reg'></a>

## **7. D'autres modèles de régression**

**À FAIRE**

>Tout est encore dans le titre. Mettez en place le modèle **de régression** que vous souhaitez.  
>Sont donc attendus dans cette partie :
>- une petite phrase pour justifier votre choix
>- les pré-traitements supplémentaires nécessaires s'il y en a
>- évaluation du modèle avec `cross_val_score` ou `cross_validate`
>- affinage des éventuels hyperparamètres avec `GridSearchCV`

#### **SVM avec différents kernels**

In [ ]:
svr_rbf = SVR(kernel='rbf', gamma=0.1)
svr_lin = SVR(kernel='linear', gamma='auto')
svr_poly = SVR(kernel='poly', gamma='auto', degree=2)

In [ ]:
svr_rbf.fit(X_train, y_train)
y_pred_svm_rbf = svr_rbf.predict(X_test)

In [ ]:
y_pred_svm_rbf.min(), y_pred_svm_rbf.max()

In [ ]:
mean_squared_error(y_pred_svm_rbf, y_test)

In [ ]:
svr_lin.fit(X_train, y_train)
y_pred_svm_lin = svr_lin.predict(X_test)

In [ ]:
y_pred_svm_lin.min(), y_pred_svm_lin.max()

In [ ]:
mean_squared_error(y_pred_svm_lin, y_test)

In [ ]:
svr_poly.fit(X_train_rfe, y_train)
y_pred_svm_poly = svr_poly.predict(X_test_rfe)

In [ ]:
y_pred_svm_poly.min(), y_pred_svm_poly.max()

In [ ]:
mean_squared_error(y_pred_svm_poly, y_test)

#### **Gradient Boosting**

In [ ]:
gradientboost = GradientBoostingRegressor(loss='ls',learning_rate=0.03,n_estimators=200,max_depth=4)
gradientboost.fit(X_train_rfe,y_train)

In [ ]:
y_pred_gb = gradientboost.predict(X_test_rfe)
error = gradientboost.loss_(y_test,y_pred_gb) # la fonction de perte est Mean square error
print("MSE:%.3f" % error)

In [ ]:
mean_squared_error(y_pred_gb, y_test)

In [ ]:
y_pred_gb.min(), y_pred_gb.max()

In [ ]:
# recherche des hyperparametres avec GridSearchCV
param_grid = {
    'loss' : ['ls'],
    'max_depth' : [6,7,8],
    'learning_rate' : [0.01],
    'n_estimators': [500, 1000, 1500]
}

grid_search_gb = GridSearchCV(GradientBoostingRegressor(), param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
grid_search_gb.fit(X_train, y_train)
grid_search_gb.best_params_

In [ ]:
grid_search_gb_pred = grid_search_gb.predict(X_test)

In [ ]:
mean_squared_error(y_test.values, grid_search_gb_pred)

#### **Forêts aléatoires**

In [ ]:
rfr = RandomForestRegressor(n_estimators = 500)
rfr.fit(X_train, y_train)
rfr_pred = rfr.predict(X_test)

In [ ]:
mean_squared_error(rfr_pred, y_test)

In [ ]:
param_grid = {
    'max_depth': [10, 50, 100, 150],
    'n_estimators': [100, 500, 1000, 1500]
}

grid_search_rfr = GridSearchCV(RandomForestRegressor(), param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
grid_search_rfr.fit(X_train, y_train)
grid_search_rfr.best_params_

In [ ]:
y_grid_pred_rfr = grid_search_rfr.predict(X_test)

In [ ]:
mean_squared_error(y_grid_pred_rfr, y_test.values)

#### **XGBoost**

In [ ]:
xgb = XGBRegressor(n_estimators = 500)
xgb.fit(X_train, y_train)

In [ ]:
mean_squared_error(xgb.predict(X_test), y_test.values)

In [ ]:
xgb.score(X_train, y_train)

In [ ]:
r2_score(y_test, xgb.predict(X_test))

In [ ]:
param_grid = {
    'max_depth': [2, 5, 10, 15],
    'learning_rate' : [0.001, 0.01, 0.1, 1],
    'n_estimators' : [500, 1000, 1500]
}

grid_search_xgb = GridSearchCV(XGBRegressor(), param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
grid_search_xgb.fit(X_train, y_train)
grid_search_xgb.best_params_

In [ ]:
xgb = XGBRegressor(n_estimators = 500)
xgb.fit(X_train, y_train)
y_pred_xgb = grid_search_xgb.predict(X_test)

In [ ]:
mean_squared_error(y_test.values, y_pred_xgb)

<a id='irr'></a>

#### **Interprétation des résultats**

On va conserver le modèle XG Boost qui obtient le plus faible MSE.

In [ ]:
feature_importance = grid_search_xgb.best_estimator_.feature_importances_
sorted_importance = np.argsort(feature_importance)
pos = np.arange(len(sorted_importance))
plt.figure(figsize=(12,12))
plt.barh(pos, feature_importance[sorted_importance],align='center')
plt.yticks(pos, X_train.columns[sorted_importance],fontsize=15)
plt.title('Feature Importance ',fontsize=18)
plt.show()

<a id='reg_to_class'></a>

## **8. De la régression à la classification**

**À FAIRE**

>Transformez le problème de régression en un problème de classification par une discrétisation du score IMDB en 5 classes : nul, bof, sympa, bon, super.  
>Justifiez votre découpage en indiquant quels seuils vous avez utilisé et pourquoi.

In [ ]:
#y_class = pd.qcut(y, 5, labels=['nul','bof','sympa','bon','super'])
y_class = pd.cut(y, [0,4,6,7,8,10], labels=[0,1,2,3,4])
y_class.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_class, test_size = 0.2, random_state=64)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_train.columns, index = X_test.index)

<a id='reglog'></a>

## **9. Une régression logistique**

**À FAIRE**

>Vous devez ici entraîner et tester une régression logistique pour la prédiction de la classe du film.  
>Par ailleurs, sont attendus ici :
>- un affichage et une interprétation des *Odds-ratio* et de leur significativité
>- le choix d'une ou plusieurs mesures d'évaluation du modèle et leur interprétation
>- une validation croisée pour l'évaluation modèle
>- l'affinage des hyperparamètres avec l'outil qui va bien
>- peut-on tracer les courbes ROC et calculer l'AUC ? Pourquoi ?

In [ ]:
reglog = LogisticRegression(solver='lbfgs')
reglog.fit(X_train,y_train)
print(reglog.score(X_train,y_train), reglog.score(X_test,y_test))

In [ ]:
sns.heatmap(confusion_matrix(reglog.predict(X_test),y_test), annot=True, fmt='d', cbar=False);

In [ ]:
# On peut aussi utiliser RFE pour sélection les variables : disons qu'on veut en garder en 25
rfe = RFE(LogisticRegression(solver='lbfgs'), 25)
rfe = rfe.fit(X_train, y_train)
col_rfe = X_train.columns[rfe.support_]

In [ ]:
#colonnes supprimées
X_train.columns[~rfe.support_]

In [ ]:
#On crée un jeu de données RFE avec uniquement les variables sélectionnées via RFE
X_train_rfe = X_train[col_rfe]
X_test_rfe = X_test[col_rfe]

In [ ]:
reglog = LogisticRegression(solver='lbfgs')
reglog.fit(X_train_rfe,y_train)
print(reglog.score(X_train_rfe,y_train), reglog.score(X_test_rfe,y_test))

<a id='autre_class'></a>

## **10. Un autre modèle de classification**

**À FAIRE**

>Au choix, une autre méthode de classification. Évidemment, sentez-vous libre d'en essayer plus d'une et de les comparer.  
>Sont donc attendus dans cette partie :
>- une petite phrase pour justifier votre choix
>- les pré-traitements supplémentaires nécessaires s'il y en a
>- évaluation du modèle
>- étude de l'importance des paramètres, si votre modèle le permet
>- affinage des éventuels hyperparamètres

#### **SVM avec différents kernels**

In [ ]:
svc_linear = SVC(kernel='linear', C=100, gamma= 'scale', decision_function_shape='ovo', random_state = 42)
svc_poly = SVC(kernel='poly', C=100, gamma= 'scale', degree = 3, decision_function_shape='ovo', random_state = 42)
svc_rbf = SVC(kernel='rbf', C=100, gamma= 'scale', decision_function_shape='ovo', random_state = 42)

In [ ]:
svc_linear.fit(X_train, y_train)
print(classification_report(y_test, svc_linear.predict(X_test)))

In [ ]:
svc_poly.fit(X_train, y_train)
print(classification_report(y_test, svc_poly.predict(X_test)))

In [ ]:
svc_rbf.fit(X_train, y_train)
print(classification_report(y_test, svc_rbf.predict(X_test)))

#### **Forêts aléatoires pour la classification**

In [ ]:
param_grid = {
    'max_depth': [50, 100, 150],
    'n_estimators': [100, 500, 1000, 1500],
    'random_state' :[0]
}

grid_search_rfc = GridSearchCV(RandomForestClassifier(), param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
grid_search_rfc.fit(X_train, y_train)
print(classification_report(y_test, grid_search_rfc.predict(X_test)))

#### **Gradient Boost pour la classification**

In [ ]:
param_grid = {
    'max_depth': [10, 50, 90],
    'max_features': [3],
    'min_samples_leaf': [3],
    'min_samples_split': [8, 10],
    'n_estimators': [100, 500],
    'learning_rate' : [0.1, 0.2],
    'random_state' : [0]
}

grid_search_gbc = GridSearchCV(GradientBoostingClassifier(), param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
grid_search_gbc.fit(X_train, y_train)
print(classification_report(y_test, grid_search_gbc.predict(X_test)))

#### **XG Boost pour la classification**

In [ ]:
param_grid = {
     'objective' : ['multi:softmax', 'multi:softprob'],
     'n_estimators': [100, 500, 1000],
     'random_state': [0]
}

grid_search_xgbc = GridSearchCV(XGBClassifier(), param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
grid_search_xgbc.fit(X_train, y_train)
print(classification_report(y_test, grid_search_xgbc.predict(X_test)))

#### **Interprétation des résultats**

On va conserver le modèle XG Boost qui obtient la meilleure *accuracy* sur l'échantillon test.

In [ ]:
feature_importance = grid_search_xgbc.best_estimator_.feature_importances_
sorted_importance = np.argsort(feature_importance)
pos = np.arange(len(sorted_importance))
plt.figure(figsize=(12,12))
plt.barh(pos, feature_importance[sorted_importance],align='center')
plt.yticks(pos, X_train.columns[sorted_importance],fontsize=15)
plt.title('Feature Importance ',fontsize=18)
plt.show()

<a id='option'></a>

## **11. En option**

Bravo, si vous êtes arrivés jusqu'ici !!!

Pour les flèches, hésitez pas à continuer si vous en voulez encore et pour les autres, hésitez pas à y revenir à l'occasion.

<a id='reco'></a>

### **11.1. Un outil de recommandation**

**À FAIRE**

>Question un peu plus ouverte pour terminer: en utilisant une méthode de clustering (donc d'apprentissage non-supervisé), construisez un petit outil de recommandation de films.  
>Pour un film donné, votre méthode doit donc retourner les films qui lui ressemblent le plus.  
>Pour rappel, on avait fait un petit exercice comme celui-cilorsqu'on avait vu les *k-plus proches voisins*, donc vous êtes invités à ne pas utiliser kNN, sinon c'est pas drôle...  
>Vous pourrez bientôt aller plus loin en créant une petite application web permettant une interface pour choisir un film.

<a id='save'></a>

### **11.2. Sauvegarder un modèle**

**À FAIRE**

>Utilisez le module `pickle` pour sauvegarder le meilleur de vos modèles et le recharger ensuite.

<a id='acp'></a>

### **11.3. Analyse en Composantes Principales**

**À FAIRE**

>Utilisez une ACP pour visualiser vos données en dimension 2 ou 3 avec des points dont la couleur varie en fonction de la classe.  